In [10]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [11]:
x = 2
y = 3

def all_calc(x,y):
    print(x + y)
    print(x * y)
    print(x - y)


all_calc(x,y)

5
6
-1
